In [2]:
import json
from dotenv import load_dotenv
from pathlib import Path

dotenv_path = Path("../../.env")
if dotenv_path.exists():
    print("Loaded .env file!")
    load_dotenv(str(dotenv_path))


data = json.load(open("/drive2/kaggle/pii-dd/data/train.json"))

Loaded .env file!


In [2]:
b_name, i_name = [], []

for d in data:
    for t, l in zip(d["tokens"], d["labels"]):
        if l == "B-NAME_STUDENT":
            b_name.append(t)
        elif l == "I-NAME_STUDENT":
            i_name.append(t)


len(b_name), len(set(b_name)), len(i_name), len(set(i_name))

(1365, 685, 1096, 643)

Many repeat names

In [3]:
# shortest is 2 letters, longest is 11/15

min([len(x) for x in b_name]), max([len(x) for x in b_name]), min([len(x) for x in i_name]), max([len(x) for x in i_name])

(2, 11, 2, 15)

In [5]:
print([i for i in set(b_name) if len(i) == 2])
print([i for i in set(i_name) if len(i) == 2])

['El', 'Jo', 'Om', 'Md', 'Fa', 'Bk', 'Sk', 'Yu', 'Al']
['Lu', 'De', 'Xu', 'M.', 'Li', 'Ka', 'Bk', 'Ng', 'Kt', 'Na', 'Kl', 'Da']


In [14]:
two_letter_context = []

for d in data:
    for i, (t, l, ws) in enumerate(zip(d["tokens"], d["labels"], d["trailing_whitespace"])):
        if (l == "B-NAME_STUDENT" or  l == "I-NAME_STUDENT") and len(t) == 2:
            start_idx = max(0, i-10)
            end_idx = min(len(d["tokens"]), i+10)

            span = ""
            for j in range(start_idx, end_idx):
                span += d["tokens"][j]
                if d["trailing_whitespace"][j]:
                    span += " "
            two_letter_context.append((t, span))

two_letter_context


[('Al',
  'find some value in it.\n\nRegards,\n\nAl\n\nMy First Mind-Mapping Session\n\nChallenge'),
 ('Lu',
  'Reflection – Stakeholder Mind Mapping\n\nBy Melvin Lu\n\nChallenge\n\nAs a Senior Manager of a '),
 ('Md', 'they need and  want.\n\nWritten by: Md Hamid\n\n'),
 ('Ka',
  'Peer Graded Assignment\n\nJoyce Ka\n\n14 December 2015\n\nChallenge  I work '),
 ('Kl',
  'Michael Kl Reflection: Visualization\n\nChallenge\n\nCompetitive organizations continuously '),
 ('Md',
  'Behavioural Change towards\n\nSanitation\n\nReflection: Storytelling\n\nMd Rashid\n\nChallenge\n\nAs part of my role '),
 ('Md',
  'Using Learning Launch\n\nChallenge and Selection\n\nI, Md Shamim, am an executive of workshop and event '),
 ('Al',
  'Design Thinking\n\nFeb 7, 2016\n\nAl Kain\n\nDesign Thinking Tools used for Aspira Diagnostics '),
 ('Jo', 'Submitted by Jo Kelly, PHR, SHRM-CP\n\nMay '),
 ('De',
  'Example of Reflection - Value Chain Analysis – Jhonatan De Oliveira\n\nChalenge & Selection\n\nI, as '

Only one has a character not in [a-zA-Z]  

`Ahmed M. Ali`

In [8]:
import re

print([x for x in set(b_name) if re.findall(r"[^a-zA-Z]", x)])
print([x for x in set(i_name) if re.findall(r"[^a-zA-Z]", x)])

[]
['M.']


# Multiple names back-to-back

It looks like there are never B-Names back to back, there are some that are 2 apart

In [11]:
dist = 1
for d in data:
    for i in range(len(d["tokens"])-dist):
        if d["labels"][i] == "B-NAME_STUDENT" and d["labels"][i+dist] == "B-NAME_STUDENT":
            print("BB")
            print([d["tokens"][i], d["tokens"][i+dist]])

        elif d["labels"][i] == "I-NAME_STUDENT" and d["labels"][i+dist] == "B-NAME_STUDENT":
            print("IB")
            print([d["tokens"][i], d["tokens"][i+dist]])

        elif d["labels"][i] == "I-NAME_STUDENT" and d["labels"][i+dist] == "I-NAME_STUDENT":
            print("II")
            print([d["tokens"][i], d["tokens"][i+dist]])

II
['Van', 'Der']
II
['Der', 'Wal']
II
['Garcia', 'Lopez']
II
['Khan', 'Pandey']
II
['Kumar', 'Aakash']
II
['Aakash', 'Kumar']
II
['Kumar', 'Aakash']
II
['Aakash', 'Kumar']
II
['Marie', 'Sanquer']
II
['Marie', 'Sanquer']
II
['Pierre', 'Jouanne']
II
['Abo', 'Fatma']
II
['Jose', 'Diaz']
II
['De', 'Oliveira']
II
['Van', 'Den']
II
['Den', 'Heuvel']
II
['Ahmed', 'Teto']
II
['Sofia', 'Rayo']
II
['Marina', 'Mora']
II
['Aakash', 'Kumar']
II
['Kumar', 'Ahmad']
II
['Kumar', 'Ahmad']
II
['Man', 'Kwok']
II
['Kumar', 'Zeeshan']
II
['Da', 'Silva']
II
['Da', 'Silva']
II
['Da', 'Silva']
II
['M.', 'Ali']
II
['Kumar', 'Ajay']
II
['Ajay', 'Rana']
II
['David', 'Medina']
II
['Sobuj', 'Islam']
II
['Luis', 'Llamas']
II
['Luis', 'Llamas']


In [12]:
dist = 2
for d in data:
    for i in range(len(d["tokens"])-dist):
        if d["labels"][i] == "B-NAME_STUDENT" and d["labels"][i+dist] == "B-NAME_STUDENT":
            print("BB")
            print([d["tokens"][i], d["tokens"][i+dist]])

        elif d["labels"][i] == "I-NAME_STUDENT" and d["labels"][i+dist] == "B-NAME_STUDENT":
            print("IB")
            print([d["tokens"][i], d["tokens"][i+dist]])

        elif d["labels"][i] == "I-NAME_STUDENT" and d["labels"][i+dist] == "I-NAME_STUDENT":
            print("II")
            print([d["tokens"][i], d["tokens"][i+dist]])

IB
['Saeed', 'Ahmed']
II
['Van', 'Wal']
IB
['Johnson', 'Katia']
IB
['Johnson', 'Katia']
IB
['Johnson', 'Denis']
BB
['Irina', 'Luca']
II
['Kumar', 'Kumar']
II
['Kumar', 'Kumar']
IB
['Chan', 'Karen']
II
['Van', 'Heuvel']
IB
['Gonzales', 'Daniel']
BB
['Naser', 'Naser']
IB
['Rajput', 'Lupe']
IB
['King', 'Jose']
IB
['Luis', 'Jose']
BB
['Stella', 'Junaid']
BB
['Stella', 'Junaid']
IB
['Birra', 'Jeff']
IB
['Carrasco', 'Jeff']
IB
['Cruz', 'Yara']
BB
['Emily', 'Giuseppe']
BB
['Giuseppe', 'Giuseppe']
BB
['Giuseppe', 'Giuseppe']
BB
['Lisa', 'Benjamin']
BB
['Kate', 'Daniel']
IB
['Huerta', 'Elisangela']
II
['Kumar', 'Rana']


There are 29 occurrences of 2 I-NAME_STUDENT back-to-back  
There are 5 occurrences of 3 I-NAME_STUDENT back-to-back-to-back

In [14]:
from collections import Counter

# consecutive I

counts = []

for d in data:
    for i in range(len(d["tokens"])):
        if d["labels"][i] == "I-NAME_STUDENT":
            temp = i
            count = 0
            while temp < len(d["tokens"]) and d["labels"][temp] == "I-NAME_STUDENT":
                temp += 1
                count += 1

            if count > 1:
                counts.append(count)

Counter(counts)

Counter({2: 29, 3: 5})